In [1]:
# основные модули
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# собственные модулей
from src.utils import prefilter_items
from src.metrics import precision_at_k, recall_at_k, recall, precision

# модель для 1го уровня
from src.recommenders import MainRecommender

# модели для 2го уровня
from lightgbm import LGBMClassifier

# отключение предупреждений
import warnings
warnings.filterwarnings('ignore')

In [2]:
# создание датафреймов
train = pd.read_csv('retail_train.csv')
test = pd.read_csv('retail_test1.csv')

# зегрузка фичей
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

In [3]:
# Снижение регистров столбцов
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

# Переименование столбцов
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [4]:
# объединение датафреймов
train_ = train.merge(item_features, on='item_id', how='left')
train_ = train.merge(user_features, on='user_id', how='left')

test_ = test.merge(item_features, on='item_id', how='left')
test_ = test.merge(user_features, on='user_id', how='left')

In [5]:
# префильтрация
train_, test_ = map(prefilter_items, (train_, test_))

In [6]:
# обучение одноуровневой модели
recommender_ = MainRecommender(train_)

In [7]:
# создание фрейма результатов одноуровневой модели
result_ = test_.groupby('user_id')['item_id'].unique().reset_index()
result_.columns = ['user_id', 'actual']

# удаление тех пользователей, на которых модель не обучалась
result_ = result_[result_['user_id'].isin(train_['user_id'])]
result_.head()

,user_id,actual
0,1,"[999999, 883616, 940947, 959219, 991024, 10049..."
1,2,"[999999, 866211, 885023, 899624, 940947, 95141..."
2,3,"[989069, 1130858]"
3,6,"[847738, 999999, 948650, 1082398, 1100159, 127..."
4,7,"[859987, 863407, 895454, 999999, 930918, 95467..."


In [8]:
result_['user_id'].unique()

array([   1,    2,    3, ..., 2498, 2499, 2500])

In [9]:
# подбор числа кандидатов


NN = [100, 150, 200, 250, 300]

for N in NN:
  
    result_['als'] = result_['user_id'].apply(lambda x: recommender_.get_als_recommendations(user=x, N=N))# apply
    result_['own'] = result_['user_id'].apply(lambda x: recommender_.get_own_recommendations(user=x, N=N))# apply
    
    als_rec = result_.apply(lambda row: recall(row['als'], row['actual']), axis=1).mean()
    own_rec = result_.apply(lambda row: recall(row['own'], row['actual']), axis=1).mean()
    
    print('Recall:')
    print('N = {}'.format(N), 'als: {}'.format(als_rec), 'own: {}'.format(own_rec))
    
 
    

Recall:
N = 100 als: 0.18146982246291427 own: 0.2344595757013779
Recall:
N = 150 als: 0.21612120639137783 own: 0.29463414596704673
Recall:
N = 200 als: 0.2440020157840414 own: 0.3371108046069097
Recall:
N = 250 als: 0.2680146216478659 own: 0.36921270294981384
Recall:
N = 300 als: 0.2913808341578366 own: 0.39275828818592945


--------------------------

In [10]:
# разбиение на тестовые и трейновые фреймы 1го и 2го уровня
weeks = 9

# 1й уровень
train_lvl1 = train_[train_['week_no'] < train_['week_no'].max() - weeks]
test_lvl1 = train_[train_['week_no'] >= train_['week_no'].max() - weeks]

# 2й уровень
train_lvl2 = test_lvl1.copy()

In [11]:
# обучение 1го уровня модели
recommender_lvl1 = MainRecommender(train_lvl1)

In [12]:
# создание фрейма результатов 1го уровня
result_lvl1 = test_lvl1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl1.columns = ['user_id', 'actual']

# удаление тех пользователей, на которых модель не обучалась
result_lvl1 = result_lvl1[result_lvl1['user_id'].isin(train_lvl1['user_id'])]
result_lvl1.head()

,user_id,actual
0,1,"[865456, 999999, 878285, 922281, 940947, 98389..."
1,2,"[999999, 839656, 866211, 893867, 1011457, 1103..."
2,3,[999999]
3,4,"[883932, 970760, 1035676, 999999, 831063, 8914..."
4,6,"[1024306, 6548453, 1098844, 999999, 8357613, 9..."


In [13]:
result_lvl1['own'] = result_lvl1['user_id'].apply(
    lambda x: recommender_lvl1.get_own_recommendations(user=x, N=200)
)# apply

In [14]:
result_lvl1

,user_id,actual,own
0,1,"[865456, 999999, 878285, 922281, 940947, 98389...","[856942, 9297615, 5577022, 9655212, 888104, 10..."
1,2,"[999999, 839656, 866211, 893867, 1011457, 1103...","[911974, 1076580, 5567582, 1103898, 1056620, 9..."
2,3,[999999],"[1092937, 1008714, 12132312, 1075979, 998206, ..."
3,4,"[883932, 970760, 1035676, 999999, 831063, 8914...","[6391541, 1052294, 891423, 936470, 1137010, 83..."
4,6,"[1024306, 6548453, 1098844, 999999, 8357613, 9...","[13003092, 972416, 995598, 923600, 1138596, 10..."
...,...,...,...
2253,2496,"[831509, 999999, 820321, 839243, 865456, 98239...","[983665, 872826, 991546, 1134296, 12452939, 74..."
2254,2497,"[824759, 999999, 965430, 970202, 977867, 98128...","[870515, 1102207, 1117219, 1057168, 1135834, 1..."
2255,2498,"[865511, 962991, 999999, 5564901, 844991, 9407...","[1022066, 1076580, 1100379, 5565356, 931579, 8..."
2256,2499,"[999999, 882308, 865992, 869322, 899624, 90412...","[7168055, 1128395, 6904613, 5570048, 830202, 8..."


In [15]:
result_lvl1.apply(lambda row: recall(row['own'], row['actual']), axis=1).mean()

0.3665939353807176

In [16]:
result_lvl1.apply(lambda row: precision_at_k(row['own'], row['actual']), axis=1).mean()

0.20896184560780834

In [17]:
# формирование фрейма с результатами прогнозирования 1го уровня для 2го уровня
result_lvl2 = pd.DataFrame(train_lvl2['user_id'].unique())
result_lvl2.columns = ['user_id']

# отбор пользователей для горячего старта
train_users = train_lvl1['user_id'].unique()
result_lvl2 = result_lvl2[result_lvl2['user_id'].isin(train_users)]

# Добавление по 100 items которые отбирает 1 уровень модели
result_lvl2['candidates'] = result_lvl2['user_id'].apply(lambda x: recommender_lvl1.get_own_recommendations(x, N=200))

result_lvl2.head()

,user_id,candidates
0,2021,"[950935, 1119454, 835578, 863762, 1019142, 102..."
1,1753,"[967041, 963686, 948640, 1057168, 942475, 9553..."
2,2120,"[5707857, 1029743, 1106523, 5569230, 916122, 8..."
3,1346,"[1135983, 5569309, 1129982, 5574377, 5569993, ..."
4,2430,"[9392700, 9803545, 1020770, 967041, 831161, 97..."


In [18]:
# формирование фрейма для 2го уровня модели

# вытаскиваем всех пользователей
users_array = result_lvl2['user_id'].values

# вытаскиваем все items
candidates_lists = result_lvl2['candidates']
len_candidates= len(candidates_lists[0])
candidates_array = candidates_lists.values

# формируем фрейм с спрогнозированными результатами
df = pd.DataFrame({'user_id':users_array.repeat(len_candidates),
                   'item_id':np.concatenate(candidates_array)})
df.head()

,user_id,item_id
0,2021,950935
1,2021,1119454
2,2021,835578
3,2021,863762
4,2021,1019142


In [19]:
# Формируем фрейм рекоммендаций user_item

# фиксируем фактическое взаимодействие
targets_train_lvl2 = train_lvl2[['user_id', 'item_id']].copy()
# отмечаем их как 1
targets_train_lvl2['target'] = 1

# объединяем фрейм фактического взаимодействия с предсказанным по совпадению пользователь-товар
targets_train_lvl2 = df.merge(targets_train_lvl2, on=['user_id', 'item_id'], how='left')
# если появятся не зафиксированные взаимодействия, отметим их как 0 
targets_train_lvl2['target'].fillna(0, inplace=True)

targets_train_lvl2

,user_id,item_id,target
0,2021,950935,0.0
1,2021,1119454,0.0
2,2021,835578,0.0
3,2021,863762,0.0
4,2021,1019142,1.0
...,...,...,...
464091,832,1000237,0.0
464092,832,904129,0.0
464093,832,909268,0.0
464094,832,823721,0.0


In [20]:
# Добавляем фичи для user, items и user_items
targets_train_lvl2 = targets_train_lvl2.merge(item_features, on='item_id', how='left')

targets_train_lvl2.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,2021,950935,0.0,2193,GROCERY,National,FRZN NOVELTIES/WTR ICE,WATER ICE,24 CT
1,2021,1119454,0.0,910,GROCERY,National,BAKED BREAD/BUNS/ROLLS,HAMBURGER BUNS,8 OZ
2,2021,835578,0.0,539,DRUG GM,National,CIGARETTES,CIGARETTES,971267 PK
3,2021,863762,0.0,6046,DRUG GM,National,PREPAID WIRELESS&ACCESSORIES,PREPAID WIRELESS CARDS,
4,2021,1019142,1.0,1007,MEAT-PCKGD,National,BREAKFAST SAUSAGE/SANDWICHES,ROLLS - FLAVORED/OTHER,1 LB


In [21]:
targets_train_lvl2 = targets_train_lvl2.merge(user_features, on='user_id', how='left')


In [22]:
targets_train_lvl2.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2021,950935,0.0,2193,GROCERY,National,FRZN NOVELTIES/WTR ICE,WATER ICE,24 CT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,1119454,0.0,910,GROCERY,National,BAKED BREAD/BUNS/ROLLS,HAMBURGER BUNS,8 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,835578,0.0,539,DRUG GM,National,CIGARETTES,CIGARETTES,971267 PK,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,863762,0.0,6046,DRUG GM,National,PREPAID WIRELESS&ACCESSORIES,PREPAID WIRELESS CARDS,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,1019142,1.0,1007,MEAT-PCKGD,National,BREAKFAST SAUSAGE/SANDWICHES,ROLLS - FLAVORED/OTHER,1 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
targets_train_lvl2[targets_train_lvl2['user_id'] == 2070]

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
94876,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
94877,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
94878,2070,879194,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,14 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
94879,2070,948640,0.0,1213,DRUG GM,National,ORAL HYGIENE PRODUCTS,WHITENING SYSTEMS,3 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
94880,2070,928263,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,13 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95076,2070,1079067,1.0,673,PRODUCE,National,ONIONS,ONIONS WHITE (BULK&BAG),5 LB,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
95077,2070,842125,0.0,1377,GROCERY,National,SHORTENING/OIL,COOKING SPRAYS,6 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
95078,2070,1070538,0.0,2,TRAVEL & LEISUR,National,APPLES,APPLES BRAEBURN (BULK&BAG),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
95079,2070,9835223,1.0,4035,MEAT,National,PORK,ENHANCED,,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [24]:
X_train = targets_train_lvl2.drop('target', axis=1)
y_train = targets_train_lvl2[['target']]

# Отмечаем категориальные признаки
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

In [25]:
lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

# прогнозирование train выборки
train_preds = lgb.predict(X_train)

train_preds

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


array([0., 0., 0., ..., 0., 0., 0.])

In [26]:
targets_test_lvl2 = test[['user_id', 'item_id']].copy()
targets_test_lvl2['target'] = 1

targets_test_lvl2 = df.merge(targets_test_lvl2, on=['user_id', 'item_id'], how='left')

targets_test_lvl2['target'].fillna(0, inplace= True)

targets_test_lvl2

,user_id,item_id,target
0,2021,950935,0.0
1,2021,1119454,0.0
2,2021,835578,0.0
3,2021,863762,0.0
4,2021,1019142,0.0
...,...,...,...
453093,832,1000237,0.0
453094,832,904129,0.0
453095,832,909268,0.0
453096,832,823721,0.0


In [27]:
targets_test_lvl2 = targets_test_lvl2.merge(item_features, on='item_id', how='left')
targets_test_lvl2 = targets_test_lvl2.merge(user_features, on='user_id', how='left')

targets_test_lvl2.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2021,950935,0.0,2193,GROCERY,National,FRZN NOVELTIES/WTR ICE,WATER ICE,24 CT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,1119454,0.0,910,GROCERY,National,BAKED BREAD/BUNS/ROLLS,HAMBURGER BUNS,8 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,835578,0.0,539,DRUG GM,National,CIGARETTES,CIGARETTES,971267 PK,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,863762,0.0,6046,DRUG GM,National,PREPAID WIRELESS&ACCESSORIES,PREPAID WIRELESS CARDS,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,1019142,0.0,1007,MEAT-PCKGD,National,BREAKFAST SAUSAGE/SANDWICHES,ROLLS - FLAVORED/OTHER,1 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
X_test = targets_test_lvl2.drop('target', axis=1)
y_test = targets_test_lvl2[['target']]

X_test[cat_feats] = X_test[cat_feats].astype('category')

In [29]:
test_preds = lgb.predict(X_test)
test_preds

array([0., 0., 0., ..., 0., 0., 0.])

In [30]:
test_preds_probability = lgb.predict_proba(X_test) 
test_preds_probability

array([[0.99105859, 0.00894141],
       [0.98374756, 0.01625244],
       [0.87729311, 0.12270689],
       ...,
       [0.96429269, 0.03570731],
       [0.97436602, 0.02563398],
       [0.97751288, 0.02248712]])

In [31]:
test_preds_probability = pd.DataFrame(test_preds_probability)

In [32]:
test_preds_probability = test_preds_probability
test_preds_probability

,0,1
0,0.991059,0.008941
1,0.983748,0.016252
2,0.877293,0.122707
3,0.945435,0.054565
4,0.884638,0.115362
...,...,...
453093,0.964189,0.035811
453094,0.984866,0.015134
453095,0.964293,0.035707
453096,0.974366,0.025634


In [33]:
X_test

,user_id,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2021,950935,2193,GROCERY,National,FRZN NOVELTIES/WTR ICE,WATER ICE,24 CT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,1119454,910,GROCERY,National,BAKED BREAD/BUNS/ROLLS,HAMBURGER BUNS,8 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,835578,539,DRUG GM,National,CIGARETTES,CIGARETTES,971267 PK,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,863762,6046,DRUG GM,National,PREPAID WIRELESS&ACCESSORIES,PREPAID WIRELESS CARDS,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,1019142,1007,MEAT-PCKGD,National,BREAKFAST SAUSAGE/SANDWICHES,ROLLS - FLAVORED/OTHER,1 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453093,832,1000237,1646,PRODUCE,National,SALAD MIX,BLENDS,10 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
453094,832,904129,69,MEAT-PCKGD,Private,MEAT - MISC,GRND/PATTY - FROZEN,3 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
453095,832,909268,1646,PRODUCE,National,SALAD MIX,BLENDS,12 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
453096,832,823721,317,GROCERY,National,CHEESE,GRATED CHEESE,8 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
final_result = X_test[['user_id', 'item_id']].copy()

In [35]:
final_result['proba'] = test_preds_probability[1]

In [36]:
final_result

,user_id,item_id,proba
0,2021,950935,0.008941
1,2021,1119454,0.016252
2,2021,835578,0.122707
3,2021,863762,0.054565
4,2021,1019142,0.115362
...,...,...,...
453093,832,1000237,0.035811
453094,832,904129,0.015134
453095,832,909268,0.035707
453096,832,823721,0.025634


In [37]:
#final_result = final_result.groupby('user_id')['proba'].nlargest(5)

In [38]:
final_result[final_result['user_id'] == 2021]

,user_id,item_id,proba
0,2021,950935,0.008941
1,2021,1119454,0.016252
2,2021,835578,0.122707
3,2021,863762,0.054565
4,2021,1019142,0.115362
...,...,...,...
195,2021,1006184,0.015858
196,2021,1082185,0.050116
197,2021,948670,0.034396
198,2021,8090440,0.029804


In [39]:
final_result = final_result.sort_values('proba', ascending=False).drop_duplicates(['user_id', 'item_id']).groupby('user_id', sort=False).head(7)

In [40]:
final_result[final_result['user_id'] == 2021]

,user_id,item_id,proba
70,2021,1029743,0.323729
39,2021,845208,0.304999
21,2021,1103808,0.301871
71,2021,1106523,0.257682
76,2021,1126899,0.206512
6,2021,1097398,0.153869
45,2021,9835223,0.150774


In [41]:
final_result = final_result.groupby('user_id')['item_id'].apply(list)

In [42]:
final_result

user_id
1       [8293439, 9655212, 1029743, 1106523, 856942, 1...
2       [1029743, 957232, 1106523, 907099, 1076580, 11...
3       [1029743, 1106523, 6534035, 1126899, 12648296,...
4       [1029743, 1106523, 977374, 1126899, 6533765, 1...
6       [1098844, 1029743, 845208, 942166, 1048257, 90...
                              ...                        
2496    [1029743, 1106523, 1070820, 1005274, 1134810, ...
2497    [854754, 12731436, 1117514, 965530, 6533936, 1...
2498    [1106523, 1070820, 1126899, 895680, 901776, 84...
2499    [1106523, 1029743, 972665, 1070820, 5592931, 6...
2500    [1019643, 859877, 1048257, 1013389, 5568758, 1...
Name: item_id, Length: 2254, dtype: object

In [43]:
final_result = pd.DataFrame(final_result)

In [44]:
final_result

,item_id
user_id,
1,"[8293439, 9655212, 1029743, 1106523, 856942, 1..."
2,"[1029743, 957232, 1106523, 907099, 1076580, 11..."
3,"[1029743, 1106523, 6534035, 1126899, 12648296,..."
4,"[1029743, 1106523, 977374, 1126899, 6533765, 1..."
6,"[1098844, 1029743, 845208, 942166, 1048257, 90..."
...,...
2496,"[1029743, 1106523, 1070820, 1005274, 1134810, ..."
2497,"[854754, 12731436, 1117514, 965530, 6533936, 1..."
2498,"[1106523, 1070820, 1126899, 895680, 901776, 84..."


In [45]:
result_

,user_id,actual,als,own
0,1,"[999999, 883616, 940947, 959219, 991024, 10049...","[883616, 962615, 1028238, 1037332, 9526630, 65...","[856942, 9297615, 5577022, 8293439, 9655212, 8..."
1,2,"[999999, 866211, 885023, 899624, 940947, 95141...","[1004906, 916122, 1029743, 919534, 866211, 844...","[1103898, 911974, 1076580, 5567582, 1007414, 1..."
2,3,"[989069, 1130858]","[1044078, 1106523, 1029743, 844179, 12810393, ...","[1092937, 1008714, 12132312, 1075979, 998206, ..."
3,6,"[847738, 999999, 948650, 1082398, 1100159, 127...","[1026118, 854852, 1084036, 1082185, 871611, 82...","[13003092, 995598, 972416, 876626, 13115971, 9..."
4,7,"[859987, 863407, 895454, 999999, 930918, 95467...","[1091365, 9803591, 839419, 1039627, 1041688, 9...","[998519, 7147142, 894360, 9338009, 993838, 896..."
...,...,...,...,...
1817,2494,"[880427, 999999, 1135006, 1135476, 823704, 829...","[8090546, 839419, 830245, 916122, 854405, 9338...","[1117219, 7024847, 1115800, 1033769, 1003524, ..."
1818,2496,"[829291, 912704, 999999, 992933, 1106523, 5591...","[844179, 899624, 1044078, 12810393, 8065410, 1...","[872826, 983665, 12452939, 991546, 1134296, 74..."
1819,2498,"[920109, 1004945, 881436, 999999]","[940947, 1045586, 948086, 830503, 1071594, 100...","[1022066, 1100379, 1076580, 931579, 5565356, 9..."
1820,2499,"[895327, 999999, 820321, 829291, 864857, 86764...","[1029743, 859154, 1054262, 5568378, 1116020, 9...","[7168055, 1128395, 6904613, 5570048, 889989, 8..."


In [46]:
#result_ = result_.merge(final_result, on=['user_id'], how='left')

In [47]:
final_result = final_result.merge(result_, on=['user_id'], how='left')

In [48]:
final_result

,user_id,item_id,actual,als,own
0,1,"[8293439, 9655212, 1029743, 1106523, 856942, 1...","[999999, 883616, 940947, 959219, 991024, 10049...","[883616, 962615, 1028238, 1037332, 9526630, 65...","[856942, 9297615, 5577022, 8293439, 9655212, 8..."
1,2,"[1029743, 957232, 1106523, 907099, 1076580, 11...","[999999, 866211, 885023, 899624, 940947, 95141...","[1004906, 916122, 1029743, 919534, 866211, 844...","[1103898, 911974, 1076580, 5567582, 1007414, 1..."
2,3,"[1029743, 1106523, 6534035, 1126899, 12648296,...","[989069, 1130858]","[1044078, 1106523, 1029743, 844179, 12810393, ...","[1092937, 1008714, 12132312, 1075979, 998206, ..."
3,4,"[1029743, 1106523, 977374, 1126899, 6533765, 1...",NaN,NaN,NaN
4,6,"[1098844, 1029743, 845208, 942166, 1048257, 90...","[847738, 999999, 948650, 1082398, 1100159, 127...","[1026118, 854852, 1084036, 1082185, 871611, 82...","[13003092, 995598, 972416, 876626, 13115971, 9..."
...,...,...,...,...,...
2249,2496,"[1029743, 1106523, 1070820, 1005274, 1134810, ...","[829291, 912704, 999999, 992933, 1106523, 5591...","[844179, 899624, 1044078, 12810393, 8065410, 1...","[872826, 983665, 12452939, 991546, 1134296, 74..."
2250,2497,"[854754, 12731436, 1117514, 965530, 6533936, 1...",NaN,NaN,NaN
2251,2498,"[1106523, 1070820, 1126899, 895680, 901776, 84...","[920109, 1004945, 881436, 999999]","[940947, 1045586, 948086, 830503, 1071594, 100...","[1022066, 1100379, 1076580, 931579, 5565356, 9..."
2252,2499,"[1106523, 1029743, 972665, 1070820, 5592931, 6...","[895327, 999999, 820321, 829291, 864857, 86764...","[1029743, 859154, 1054262, 5568378, 1116020, 9...","[7168055, 1128395, 6904613, 5570048, 889989, 8..."


In [50]:
final_result.apply(lambda row: precision_at_k(row['item_id'], row['actual']), axis=1).mean()

0.07684117125110913